In [1]:
import logging
import sys
import os # Necesario para os.path.exists

# Configuración básica de logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import Settings, StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.prompts import PromptTemplate
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler, CBEventType

/home/daniel/PROGRAMAS/miniconda3/envs/CHILEAN_AC/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/daniel/PROGRAMAS/miniconda3/envs/CHILEAN_AC/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Callback Handler Personalizado para capturar el prompt ---
class PromptCaptureHandler(LlamaDebugHandler):
    def __init__(self):
        super().__init__()
        self.last_llm_prompt = None
        self.last_llm_inputs = None

    def on_event_start(
        self,
        event_type: CBEventType,
        payload: dict = None,
        event_id: str = "",
        parent_id: str = "",
        **kwargs,
    ) -> str:
        if event_type == CBEventType.LLM:
            # El payload en el evento LLM contiene los argumentos pasados al LLM
            # Esto incluye los mensajes o el prompt formateado
            if payload:
                # Para modelos de chat, los prompts están en 'messages'
                if "messages" in payload:
                    self.last_llm_inputs = payload.get("messages")
                # Para modelos de completado, puede estar en una lista de 'prompts'
                elif "prompts" in payload:
                     self.last_llm_inputs = payload.get("prompts")

        return event_id # Devuelve el event_id

    def on_event_end(
        self,
        event_type: CBEventType,
        payload: dict = None,
        event_id: str = "",
        **kwargs,
    ) -> None:
        # Aquí podrías capturar la respuesta del LLM si quisieras
        pass

    def get_last_llm_prompt_str(self):
        if not self.last_llm_inputs:
            return "No se capturó ningún prompt para el LLM."

        # Si son mensajes de chat (lo más común ahora)
        if isinstance(self.last_llm_inputs, list) and hasattr(self.last_llm_inputs[0], 'content'):
            return "\n".join([f"Role: {msg.role}\nContent: {msg.content}" for msg in self.last_llm_inputs])
        # Si es una lista de strings de prompt (para modelos de completado más antiguos)
        elif isinstance(self.last_llm_inputs, list) and isinstance(self.last_llm_inputs[0], str):
            return "\n---\n".join(self.last_llm_inputs)
        return str(self.last_llm_inputs)

In [3]:


# --- 1. Configurar Componentes Globales (LLM y Modelo de Embeddings) ---
print("Configurando LLM, Embeddings y Callback Manager...")
prompt_capture_handler = PromptCaptureHandler()
callback_manager = CallbackManager([prompt_capture_handler])
try:
    Settings.llm = Ollama(model="qwen3:1.7b-q8_0", request_timeout=3600.0) # o tu modelo preferido
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    Settings.callback_manager = callback_manager # Asignar el callback manager globalmente
    print("LLM y Modelo de Embeddings configurados.")
except Exception as e:
    print(f"Error configurando LLM o Embeddings: {e}")
    exit()


Configurando LLM, Embeddings y Callback Manager...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
LLM y Modelo de Embeddings configurados.


In [4]:

# --- 2. Cargar el Índice Vectorial Persistido ---
persist_directory = "./unfair_tos_kb_index_train_val" # El mismo directorio donde guardaste el índice

if not os.path.exists(persist_directory):
    print(f"Error: El directorio del índice persistido '{persist_directory}' no fue encontrado.")
    print("Asegúrate de haber ejecutado el script anterior para crear y persistir el índice.")
    exit()

print(f"\nCargando el índice desde: {persist_directory}...")
try:
    storage_context = StorageContext.from_defaults(persist_dir=persist_directory)
    index = load_index_from_storage(storage_context)
    print("Índice cargado exitosamente.")
except Exception as e:
    print(f"Error cargando el índice: {e}")
    exit()


Cargando el índice desde: ./unfair_tos_kb_index_train_val...
INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********
Índice cargado exitosamente.


In [5]:
# --- 3. Crear un Motor de Consulta (Query Engine) ---
# El Query Engine se encarga de la lógica RAG: recuperar y luego sintetizar.
print("\nCreando el motor de consulta...")
query_engine = index.as_query_engine(
    similarity_top_k=3, # Recuperar los 3 chunks más similares
    # response_mode="compact" # Puedes experimentar con diferentes modos de respuesta
)
print("Motor de consulta creado.")


Creando el motor de consulta...
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
Motor de consulta creado.


In [6]:
# --- 4. Definir la Cláusula a Clasificar y el Prompt de Clasificación ---

# Nombres de las etiquetas en el ORDEN CORRECTO
label_names_ordered = [
    'Limitation of liability', # Índice 0
    'Unilateral termination',  # Índice 1
    'Unilateral change',       # Índice 2
    'Content removal',         # Índice 3
    'Contract by using',       # Índice 4
    'Choice of law',           # Índice 5
    'Jurisdiction',            # Índice 6
    'Arbitration'              # Índice 7
]
# Crear una representación de las etiquetas para el prompt
etiquetas_posibles_str = "\n".join([f"- {name}" for name in label_names_ordered])



# Crear un prompt de plantilla para la tarea de clasificación multietiqueta
# Esto es un ejemplo. El "prompt engineering" es clave aquí.
# Le pedimos al LLM que use el contexto para identificar las etiquetas.
classification_prompt_template = PromptTemplate(
    "Eres un asistente legal experto en identificar cláusulas abusivas en Términos de Servicio.\n"
    "Analiza la siguiente Cláusula Nueva basándote en el Contexto Proporcionado (cláusulas similares y sus etiquetas) y determina a cuáles de las siguientes categorías de cláusulas abusivas pertenece.\n"
    "Responde ÚNICAMENTE con los nombres de las categorías de la lista que apliquen, separados por comas. Si ninguna aplica, responde 'Ninguna'.\n\n"
    "Categorías Posibles:\n"
    f"{etiquetas_posibles_str}\n\n"
    "Contexto Proporcionado (Cláusulas Similares de la Base de Conocimiento y sus Etiquetas):\n"
    "---------------------\n"
    "{context_str}\n"  # LlamaIndex llenará esto con los chunks recuperados
    "---------------------\n"
    "Cláusula Nueva a Clasificar:\n"
    "{query_str}\n\n"  # LlamaIndex llenará esto con la cláusula_a_clasificar
    "Etiquetas Identificadas (separadas por comas o 'Ninguna'):"
)

# Actualizar el prompt en el query_engine
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": classification_prompt_template}
)
print("Prompt de clasificación actualizado en el motor de consulta.")


Prompt de clasificación actualizado en el motor de consulta.


In [7]:
# Cláusula de ejemplo que queremos clasificar (puedes cambiarla)
clausula_a_clasificar = "amazon reserves the right to refuse service , terminate accounts , terminate your rights to use amazon services , remove or edit content , or cancel orders in its sole discretion."

In [8]:
# --- 5. Realizar la Consulta RAG para Clasificación ---
print(f"\nClasificando la cláusula: \"{clausula_a_clasificar}\"")
try:
    respuesta_rag = query_engine.query(clausula_a_clasificar)

    print("\n--- Prompt Final Enviado al LLM (Capturado por Callback) ---")
    print(prompt_capture_handler.get_last_llm_prompt_str()) # Imprimir el prompt capturado

    print("\n--- Respuesta del LLM (Clasificación Sugerida) ---")
    print(respuesta_rag.response)

    print("\n--- Nodos Fuente Utilizados para la Clasificación (Contexto) ---")
    for i, source_node in enumerate(respuesta_rag.source_nodes):
        print(f"Nodo {i+1} (Score de similitud: {source_node.score:.4f}):")
        print(f"Texto: {source_node.get_content()[:150]}...")
        print(f"Metadatos originales: {source_node.metadata}") # Aquí verás las etiquetas originales de este chunk
        print("-" * 20)

except Exception as e:
    print(f"Error durante la consulta RAG para clasificación: {e}")

print("\n--- Fin del Ejemplo RAG para Clasificación con LlamaIndex ---")


Clasificando la cláusula: "amazon reserves the right to refuse service , terminate accounts , terminate your rights to use amazon services , remove or edit content , or cancel orders in its sole discretion."
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
**********
Trace: query
**********

--- Prompt Final Enviado al LLM (Capturado por Callback) ---
Role: MessageRole.USER
Content: Eres un asistente legal experto en identificar cláusulas abusivas en Términos de Servicio.
Analiza la siguiente Cláusula Nueva basándote en el Contexto Proporcionado (cláusulas similares y sus etiquetas) y determina a cuáles de las siguientes categorías de cláusulas abusivas pertenece.
Responde ÚNICAMENTE con los nombres de las categorías de la lista que apliquen, separados por comas. Si ninguna aplica, responde 'Ninguna'.

Categorías Posibles:
- Limitation of liability
- Unilateral termination
- Unilateral 

In [9]:
# --- Funciones Auxiliares ---
import re

def parse_space_separated_label_list(label_str):
    if pd.isna(label_str) or not isinstance(label_str, str):
        return []
    cleaned_str = label_str.strip().strip('[]').strip()
    if not cleaned_str:
        return []
    try:
        return [int(num_str) for num_str in cleaned_str.split()]
    except ValueError:
        return []

def extract_reasoning_and_prediction(llm_full_response: str):
    """
    Extrae el razonamiento (dentro de <think></think>) y la predicción final.
    """
    reasoning = None
    prediction_str = llm_full_response # Por defecto, toda la respuesta es la predicción

    think_match = re.search(r"<think>(.*?)</think>(.*)", llm_full_response, re.DOTALL | re.IGNORECASE)

    if think_match:
        reasoning = think_match.group(1).strip()
        prediction_str = think_match.group(2).strip()

    # Si después de quitar el bloque <think>, la predicción está vacía,
    # o si no hubo bloque <think> pero la respuesta es solo "Ninguna" u otra etiqueta,
    # la tomamos como la predicción.
    # Si la predicción_str sigue conteniendo saltos de línea inesperados o texto residual,
    # podríamos necesitar una lógica más robusta para aislar la línea final de etiquetas.
    # Por ahora, asumimos que la predicción está justo después de </think> o es toda la respuesta.

    # Un intento simple de tomar la última línea no vacía como la predicción si hay múltiples líneas después de </think>
    potential_prediction_lines = [line.strip() for line in prediction_str.split('\n') if line.strip()]
    if potential_prediction_lines:
        prediction_str = potential_prediction_lines[-1]

    return reasoning, prediction_str


def parse_llm_prediction_to_indices(prediction_final_str: str, label_names_ordered: list) -> list:
    """Convierte la cadena de predicción final del LLM a una lista de índices."""
    if prediction_final_str.strip().lower() == 'ninguna':
        return []

    predicted_label_names = [name.strip().lower() for name in prediction_final_str.split(',')]
    indices = []
    for i, official_name in enumerate(label_names_ordered):
        if official_name.lower() in predicted_label_names:
            indices.append(i)
    return sorted(list(set(indices)))

def convert_indices_to_multihot(label_indices: list, num_total_labels: int) -> list:
    """Convierte una lista de índices a un vector multi-hot."""
    multihot = [0] * num_total_labels
    for idx in label_indices:
        if 0 <= idx < num_total_labels:
            multihot[idx] = 1
    return multihot

In [10]:
from tqdm import tqdm
import pandas as pd
NUM_LABELS = len(label_names_ordered)
# --- 4. Cargar y Preparar el Dataset de Test ---
output_dir = "../unfair_tos_data"
test_file_path = os.path.join(output_dir, "unfair_tos_test.csv")

if not os.path.exists(test_file_path):
    print(f"Error: El archivo de test '{test_file_path}' no fue encontrado.")
    exit()

print(f"\nCargando el archivo de test: {test_file_path}...")
df_test = pd.read_csv(test_file_path)
df_test['true_labels_indices'] = df_test['labels'].apply(parse_space_separated_label_list)
print(f"Archivo de test cargado. Total de filas: {len(df_test)}")

# Seleccionar las primeras N muestras para clasificar
num_muestras_a_clasificar = 3200
df_test_sample = df_test.head(num_muestras_a_clasificar).copy() # Usar .copy() para evitar SettingWithCopyWarning
print(f"Se clasificarán las primeras {len(df_test_sample)} muestras del archivo de test.")


Cargando el archivo de test: unfair_tos_data/unfair_tos_test.csv...
Archivo de test cargado. Total de filas: 1607
Se clasificarán las primeras 1607 muestras del archivo de test.


In [11]:
df_test_sample

,text,labels,true_labels_indices
0,"last updated date : may 15 , 2017 \n",[],[]
1,"academia , inc. ( `` academia.edu '' or `` we ...",[],[]
2,please read carefully the following terms and ...,[],[]
3,these terms govern your access to and use of t...,[],[]
4,arbitration notice : unless you opt out of arb...,[7],[7]
...,...,...,...
1602,"code § 1789.3 , you may report complaints to t...",[],[]
1603,"pursuant to 815 ilcs 414/1 .5 ( c ) , for tran...",[],[]
1604,the aaa 's rules are available at www.adr.org ...,[],[]
1605,such complaints shall be decided by an indepen...,[],[]


In [ ]:
print("\nIniciando clasificación RAG para las muestras de test...")
llm_full_responses = []
llm_reasonings = []
predicted_final_labels_str = []
predicted_indices_list = []
true_indices_list = []

for index_df, row in tqdm(df_test_sample.iterrows(), total=len(df_test_sample), desc="Clasificando"):
    clausula_a_clasificar = str(row['text'])
    true_labels_for_sample = row['true_labels_indices']
    true_indices_list.append(true_labels_for_sample)

    try:
        respuesta_rag = query_engine.query(clausula_a_clasificar)
        full_response_str = respuesta_rag.response.strip()
        llm_full_responses.append(full_response_str) # Guardar la respuesta completa

        # Extraer razonamiento y predicción final
        reasoning, prediction_final_str = extract_reasoning_and_prediction(full_response_str)

        llm_reasonings.append(reasoning)
        predicted_final_labels_str.append(prediction_final_str)

        predicted_label_indices = parse_llm_prediction_to_indices(prediction_final_str, label_names_ordered)
        predicted_indices_list.append(predicted_label_indices)

    except Exception as e:
        print(f"Error clasificando la cláusula ID {row.get('id', index_df)}: {e}")
        llm_full_responses.append("ERROR_EN_GENERACION")
        llm_reasonings.append("ERROR_EN_GENERACION")
        predicted_final_labels_str.append("ERROR_EN_GENERACION")
        predicted_indices_list.append([])

df_test_sample['llm_full_response'] = llm_full_responses
df_test_sample['llm_reasoning'] = llm_reasonings
df_test_sample['predicted_final_labels_str'] = predicted_final_labels_str # Solo la parte de etiquetas
df_test_sample['predicted_labels_indices'] = predicted_indices_list

print("Clasificación RAG completada.")


Iniciando clasificación RAG para las muestras de test...


Clasificando:   0%|          | 0/1607 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
**********
Trace: query
**********


Clasificando:   0%|          | 1/1607 [00:09<4:17:35,  9.62s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
**********
Trace: query
**********


...

Clasificando:   2%|▏         | 32/1607 [07:53<5:49:52, 13.33s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
**********
Trace: query
**********


Clasificando:   2%|▏         | 33/1607 [08:03<5:25:41, 12.42s/it]

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
**********
Trace: query
**********


Clasificando:   2%|▏         | 34/1607 [08:23<6:22:38, 14.60s/it]

In [29]:
# --- 6. Generar y Mostrar el Reporte ---
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, classification_report

print("\n--- Reporte de Clasificación (Primeras Muestras de Test) ---")

y_true_multihot = [convert_indices_to_multihot(lst, NUM_LABELS) for lst in df_test_sample['true_labels_indices']]
y_pred_multihot = [convert_indices_to_multihot(lst, NUM_LABELS) for lst in df_test_sample['predicted_labels_indices']]

print("\nEjemplos de Predicciones (con Razonamiento):")
for i in range(min(10, len(df_test_sample))):
    true_names = [label_names_ordered[idx] for idx in df_test_sample['true_labels_indices'].iloc[i]]
    reasoning_text = df_test_sample['llm_reasoning'].iloc[i]
    pred_final_str = df_test_sample['predicted_final_labels_str'].iloc[i]

    clause_id_to_print = df_test_sample['id'].iloc[i] if 'id' in df_test_sample.columns else f"Fila_{df_test_sample.index[i]}"

    print(f"Cláusula {i+1} ID: {clause_id_to_print}")
    print(f"  Texto: {df_test_sample['text'].iloc[i][:100]}...")
    print(f"  Etiquetas Verdaderas: {true_names if true_names else 'Ninguna'}")
    if reasoning_text:
        print(f"  Razonamiento del LLM:\n{reasoning_text[:300]}...\n") # Mostrar solo una parte del razonamiento
    print(f"  Predicción Final del LLM (Etiquetas): {pred_final_str}")
    print("-" * 20)

# Reporte de Clasificación de Scikit-learn
# Nota: Para multilabel, target_names debe corresponder a las columnas del y_true_multihot/y_pred_multihot
# que son los índices de 0 a NUM_LABELS-1. Los nombres se usarán para mostrar el reporte.
try:
    report = classification_report(
        y_true_multihot,
        y_pred_multihot,
        target_names=label_names_ordered,
        zero_division=0
    )
    print("\nReporte de Clasificación General (scikit-learn):")
    print(report)
except ValueError as ve:
    print(f"\nError generando el classification_report: {ve}")
    # ... (cálculo de métricas individuales como antes) ...
    micro_f1 = f1_score(y_true_multihot, y_pred_multihot, average='micro', zero_division=0)
    macro_f1 = f1_score(y_true_multihot, y_pred_multihot, average='macro', zero_division=0)
    print(f"  Micro F1-score: {micro_f1:.4f}")
    print(f"  Macro F1-score: {macro_f1:.4f}")

exact_match_ratio = sum(
    1 for i_match in range(len(y_true_multihot)) if y_true_multihot[i_match] == y_pred_multihot[i_match]
) / len(y_true_multihot)
print(f"\nExact Match Ratio (Precisión de Subconjunto de Etiquetas): {exact_match_ratio:.4f}")

# Opcional: Guardar el DataFrame con los resultados para análisis posterior
df_test_sample.to_csv(os.path.join(output_dir, "test_sample_predictions_with_reasoning.csv"), index=False)
print(f"\nResultados detallados (incluyendo razonamiento) guardados en: {os.path.join(output_dir, 'test_sample_predictions_with_reasoning.csv')}")

print("\n--- Fin del Script ---")


--- Reporte de Clasificación (Primeras Muestras de Test) ---

Ejemplos de Predicciones (con Razonamiento):
Cláusula 1 ID: Fila_0
  Texto: last updated date : may 15 , 2017 
...
  Etiquetas Verdaderas: Ninguna
  Razonamiento del LLM:
Okay, let's tackle this query. The user wants me to determine which categories of abusive clauses the given clause falls into. The clause in question is "last updated date : may 15 , 2017 ".

First, I need to look at the provided context. There are three existing clauses. The first one mentions the ...

  Predicción Final del LLM (Etiquetas): Ninguna
--------------------
Cláusula 2 ID: Fila_1
  Texto: academia , inc. ( `` academia.edu '' or `` we '' ) offers a social networking service which enables ...
  Etiquetas Verdaderas: Ninguna
  Razonamiento del LLM:
Okay, let's tackle this query. The user wants me to determine which categories of abusive clauses the given new clause falls into. The categories are Limitation of liability, Unilateral termination, Un